# pip, import and kaggle

In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
! pip install kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
! mkdir ~/.kaggle

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [4]:
! cp kaggle.json ~/.kaggle/

In [5]:
! chmod 600 ~/.kaggle/kaggle.json

In [6]:
! kaggle datasets download burhanbilenn/duygu-analizi-icin-urun-yorumlari

duygu-analizi-icin-urun-yorumlari.zip: Skipping, found more recently modified local copy (use --force to force download)


In [7]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import torch
from transformers import AutoModel, AutoTokenizer

# sentences & label: change index for run

In [8]:
!unzip /content/duygu-analizi-icin-urun-yorumlari.zip

Archive:  /content/duygu-analizi-icin-urun-yorumlari.zip
replace magaza_yorumlari_duygu_analizi.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [9]:
df = pd.read_csv('/content/magaza_yorumlari_duygu_analizi.csv', encoding='utf-16')
df_wo_Tarafsiz = df[df['Durum'] != 'Tarafsız']
df_wo_Tarafsiz.loc[:,'encoded_Durum'] = LabelEncoder().fit_transform(df_wo_Tarafsiz.loc[:,'Durum'])

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value


In [10]:
df_run = df_wo_Tarafsiz[7000:8000] # 8000 total
sentences = df_run['Görüş'].dropna().values
labels = df_run['encoded_Durum'].values

# berturk

In [11]:
max_seq_len=50

tokenizer = AutoTokenizer.from_pretrained("dbmdz/bert-base-turkish-uncased")
model = AutoModel.from_pretrained("dbmdz/bert-base-turkish-uncased")

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# device = torch.device('cpu')
model.to(device)

Some weights of the model checkpoint at dbmdz/bert-base-turkish-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(32000, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [12]:
input_ids = []
for text in sentences:
    encoded_dict = tokenizer.encode_plus(text, add_special_tokens = True, max_length = max_seq_len, truncation=True,
                                          pad_to_max_length = True, return_tensors = 'pt')
    input_ids.append(encoded_dict['input_ids'])
input_ids = torch.cat(input_ids, dim=0).to(device)
labels = torch.tensor(labels)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2329: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [13]:
representations = []
for input in input_ids:
  cls = model(input.unsqueeze(0))['last_hidden_state'][0][0]
  representations.append(cls)

rs_torch = torch.stack(representations)

rs_numpy = rs_torch.detach().numpy()
labels_numpy = labels.numpy()

np.save('/content/rs_run7', rs_numpy)
np.save('/content/labels_run7', labels_numpy)